In [1]:
import os
import pandas as pd
import numpy as np
import re
import math
import string
import requests
from itertools import product
from inspect import getsourcefile

os.chdir('C:\\Users\\mwa150')
pd.set_option('display.max_colwidth', -1)

# text cleaning for tweets

In [17]:
import string
from html.parser import HTMLParser
import re
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup

split_adhere_dic = { # split
                    "cause": "because", "could've": "could have", 
                    "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                    "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                    "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                    "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                    "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                    "i'll've": "i will have","i'm": "i am", "i've": "i have",
                    "it'd": "it would", "it'd've": "it would have", "it'll": "it will", 
                    "it'll've": "it will have","it's": "it is", 
                    "let's": "let us", "ma'am": "madam", "might've": "might have",
                    "o'clock": "of the clock", 
                    "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                    "she'll've": "she will have", "she's": "she is", "should've": "should have",
                    "so've": "so have","so's": "so as", 
                    "this's": "this is",
                    "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                    "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                    "here's": "here is",
                    "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                    "they'll've": "they will have", "they're": "they are", "they've": "they have",
                    "to've": "to have", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", 
                    "we'll've": "we will have", "we're": "we are", "we've": "we have",
                    "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                    "what's": "what is", "what've": "what have", "when's": "when is", 
                    "when've": "when have", "where'd": "where did", "where's": "where is", 
                    "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                    "who's": "who is", "who've": "who have", "why's": "why is", 
                    "why've": "why have", "will've": "will have", "would've": "would have",
                    "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
                    "y'all're": "you all are","y'all've": "you all have", "you'd": "you would", 
                    "you'd've": "you would have", "you'll": "you will", 
                    "you'll've": "you will have", "you're": "you are", "you've": "you have",
                     # adhere 
                    "is not": "ain't", "are not": "aren't","cannot": "can't", "can not": "can't", 
                    "cannot have": "can't've", "could not": "couldn't", "could not have": "couldn't've","did not": "didn't", 
                    "does not": "doesn't", "do not": "don't", "had not": "hadn't",  
                    "had not have": "hadn't've", "has not": "hasn't", "have not": "haven't", "is not": "isn't", 
                    "may not": "mayn't", "might not": "mightn't","might not have": "mightn't've", 
                    "must have": "must've", "must not": "mustn't", "must not have": "mustn't've", 
                    "need not": "needn't", "need not have": "needn't've",
                    "ought not": "oughtn't", "ought not have": "oughtn't've", "shall not": "shan't",
                    "shall not": "sha'n't", "shall not have": "shan't've", 
                    "should not": "shouldn't", "should not have": "shouldn't've", "was not": "wasn't", "were not": "weren't",
                    "will not": "won't", "will not have": "won't've",  "would not": "wouldn't", 
                    "would not have": "wouldn't've", 
                    "no prblme": "noproblem", "no prob": "noprob", "no problems": "noproblmes", "no probs": "noprobs",
                    "no worry": "noworry", "no worries": "noworries", "good bey": "goodbey", "long time no see": "longtimenosee", 
                    "nothing but": "nothingbut", "no more": "nomore","kind of": "kindof", "sort of": "sortof"}
    
pat1 = r'@[\w_]+' # @-mention
pat2 = r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+' # URLs
pat3 = r'#[\w_]+' # hash-tags
pat5 = r'www.[^ ]+' # additions to URLs, texts with 'www..'
combined_pat = r'|'.join((pat1, pat2, pat3, pat5))

split_pattern = re.compile(r'\b(' + '|'.join(split_adhere_dic.keys()) + r')\b')

def tweet_cleaner(demo):
    soup = BeautifulSoup(demo, 'lxml') # HTML
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    split_handled = split_pattern.sub(lambda x: split_adhere_dic[x.group()], stripped)

    return split_handled

In [53]:
df = pd.read_csv('politicians_vader.csv')
df = df.head(5)

# apply tweet_cleaner to tweets
df['new_clean_text'] = [tweet_cleaner(t) for t in df.full_text]
# remove punctuations
df['new_clean_text'] = df['new_clean_text'].str.replace(r'[^\w\s]+', '')
# and add '.' to the end of each line
df['new_clean_text'] = df['new_clean_text'] + "."
    
text = df['new_clean_text'].to_string()

# transfer clean_text to str by txt
# for c in df.columns:
#     df['new_clean_text'].to_csv(c + '.txt', index=False)
    
# with open('new_clean_text.txt', 'r') as file:
#     data = file.read().replace('\n', '')

C:\Program Files\Anaconda\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\Program Files\Anaconda\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\Program Files\Anaconda\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\Program Files\Anaconda\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
C:\Program Files\Anaconda\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  

# IBM Tone Analyzer

In [55]:
from watson_developer_cloud import ToneAnalyzerV3

tone_analyzer = ToneAnalyzerV3(
    version='',
    iam_apikey='',
    url=''
)

# sample_text = '0 Team, I know that times are tough!  '\
#    '1 Sales have been disappointing for the past three. '\
#    '2 I need to do a better job of selling it!'
    
text = text

tone_analysis = tone_analyzer.tone(
    {'text': text},
    'application/json').get_result()

print(json.dumps(tone_analysis, indent=2))

C:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: ToneAnalyzerV3 is a deprecated function. watson-developer-cloud moved to ibm-watson
  if __name__ == '__main__':


{
  "document_tone": {
    "tones": [
      {
        "score": 0.70704,
        "tone_id": "joy",
        "tone_name": "Joy"
      },
      {
        "score": 0.693882,
        "tone_id": "analytical",
        "tone_name": "Analytical"
      }
    ]
  },
  "sentences_tone": [
    {
      "sentence_id": 0,
      "text": "0    Roger Stones attorney sent a letter this week stating Stone wont provide documents or appear for an interview before the committee  .                                                                                                      ",
      "tones": [
        {
          "score": 0.659112,
          "tone_id": "tentative",
          "tone_name": "Tentative"
        }
      ]
    },
    {
      "sentence_id": 1,
      "text": "1    Education Secretary DeVos proposal to rewrite campus sexual assault guidelines will harm survivors and likely discourage many from ever coming forward Leave a comment and tell the Education Dept we dont want these protections gutted  .

In [105]:
# save as json file
import json

with open('data.json', 'w') as fp:
    json.dump(tone_analysis, fp, indent=4)    

In [147]:
data = json.load(open('data.json'))
df = pd.DataFrame(data["sentences_tone"])
df

,sentence_id,text,tones
0,0,0 Roger Stones attorney sent a letter this week stating Stone wont provide documents or appear for an interview before the committee .,"[{'score': 0.659112, 'tone_id': 'tentative', 'tone_name': 'Tentative'}]"
1,1,1 Education Secretary DeVos proposal to rewrite campus sexual assault guidelines will harm survivors and likely discourage many from ever coming forward Leave a comment and tell the Education Dept we dont want these protections gutted .,"[{'score': 0.52354, 'tone_id': 'sadness', 'tone_name': 'Sadness'}, {'score': 0.620279, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]"
2,2,2 US citizen children are eligible for health insurance coverage but President Trumps plan to bar legal immigrants from green cards is keeping immigrant parents from signing them up .,[]
3,3,3 Happy to all those celebrating in California and across the world I hope everyone enjoys a Festival of Lights surrounded by friends and family.,"[{'score': 0.943498, 'tone_id': 'joy', 'tone_name': 'Joy'}]"
4,4,4 Theres no question that George HW Bush led our country through a time of turmoil The fall of the Soviet Union and the Gulf War were both defining moments that continue to shape our nation today.,"[{'score': 0.794465, 'tone_id': 'analytical', 'tone_name': 'Analytical'}]"


In [ ]:
add_score = []
add_toneid = []

for index, row in df.iterrows():
    sc = (row['tones']['score'])
    to = (row['tones']['tone_name'])
    
    add_score.append(sc)
    add_toneid.append(to)
    
df['tone_score'] = add_score
df['tone_id'] = add_toneid